This notebook uses a BERT-based model to predict the 'M' label of the TNM staging classification.

In [1]:
import os
import pickle
from datetime import timedelta
import numpy as np
import pandas as pd
import time
import copy

import sys
sys.path.append('..')
import utils
import llm_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import torch
import torchinfo
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, set_seed


In [2]:
# Constants and arguments
seq_len = 4096
epochs = 10
lr = 2e-5
bs = 6
cuda_gpu_id = "1"

tnm_label = 'm'

model_name = "yikuan8/Clinical-BigBird"
data_dir = "../../data/tnm_stage"
out_path = f"./model_weights_{tnm_label}"
out_preds_path = "./model_preds"

In [3]:
if cuda_gpu_id != "-1":
    os.environ["CUDA_VISIBLE_DEVICES"] = cuda_gpu_id
_ = torch.device('cuda')
torch.backends.cuda.matmul.allow_tf32 = True
assert torch.cuda.is_available()
print("Number of GPUs available:", torch.cuda.device_count())

Number of GPUs available: 1


# Data loading

In [4]:
label_enc = LabelEncoder()

## Training

In [5]:
df_train = pd.read_csv(os.path.join(data_dir, "train_tcga_reports_tnm_stage.csv"))

In [6]:
df_train.shape

(1947, 6)

In [7]:
df_train[f'{tnm_label}_label'].value_counts()

m_label
M0    1821
M1     126
Name: count, dtype: int64

In [8]:
df_train[f'{tnm_label}_class'] = label_enc.fit_transform(df_train[f'{tnm_label}_label'])

## Validation

In [9]:
df_val = pd.read_csv(os.path.join(data_dir, "val_tcga_reports_tnm_stage.csv"))

In [10]:
df_val.shape

(780, 6)

In [11]:
df_val[f'{tnm_label}_class'] = label_enc.fit_transform(df_val[f'{tnm_label}_label'])

## Test

In [12]:
df_test = pd.read_csv(os.path.join(data_dir, "test_tcga_reports_tnm_stage.csv"))

In [13]:
df_test.shape

(1170, 6)

In [14]:
df_test[f'{tnm_label}_class'] = label_enc.fit_transform(df_test[f'{tnm_label}_label'])

# Model training

## Tokenization

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/common/lopezgg/miniconda3/envs/nlp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
arr_train_text = df_train['text'].apply(str).to_list()
arr_train_label = df_train[f'{tnm_label}_class'].apply(int).to_list()

arr_val_text = df_val['text'].apply(str).to_list()
arr_val_label = df_val[f'{tnm_label}_class'].apply(int).to_list()

arr_test_text = df_test['text'].apply(str).to_list()
arr_test_label = df_test[f'{tnm_label}_class'].apply(int).to_list()

We first analyze the token length of each document in the corpus:

In [17]:
arr_corpus_text = arr_train_text + arr_val_text + arr_test_text
arr_tok = []
for document in arr_corpus_text:
    tokens = tokenizer(
        document,
        truncation=False,
        padding=False
    )
    arr_tok.append(tokens['input_ids'])

Token indices sequence length is longer than the specified maximum sequence length for this model (5074 > 4096). Running this sequence through the model will result in indexing errors


In [18]:
arr_tok_len = pd.Series([len(seq) for seq in arr_tok])
print(arr_tok_len.describe())

count    3897.000000
mean      877.958686
std       824.078881
min        27.000000
25%       242.000000
50%       634.000000
75%      1242.000000
max      5447.000000
dtype: float64


In [19]:
print(pd.DataFrame({
    "abs": (arr_tok_len <= seq_len).value_counts(normalize=False),
    "rel": (arr_tok_len <= seq_len).value_counts(normalize=True)
}))
print()

        abs       rel
True   3877  0.994868
False    20  0.005132



Only 20 documents do not fit into the model.

In [20]:
train_encodings = tokenizer(
    arr_train_text,
    truncation=True,
    padding=True,
    max_length=seq_len,
    return_tensors="pt"
)

In [21]:
val_encodings = tokenizer(
    arr_val_text,
    truncation=True,
    padding=True,
    max_length=seq_len,
    return_tensors="pt"
)

In [22]:
test_encodings = tokenizer(
    arr_test_text,
    truncation=True,
    padding=True,
    max_length=seq_len,
    return_tensors="pt"
)

In [23]:
train_dataset = llm_utils.CustomDataset(
    encodings=train_encodings,
    labels=torch.tensor(arr_train_label)
)

In [24]:
val_dataset = llm_utils.CustomDataset(
    encodings=val_encodings,
    labels=torch.tensor(arr_val_label)
)

In [25]:
test_dataset = llm_utils.CustomDataset(
    encodings=test_encodings,
    labels=torch.tensor(arr_test_label)
)

In [26]:
print("Train data length:", len(train_dataset))
print("Val data length:", len(val_dataset))
print("Test data length:", len(test_dataset))

Train data length: 1947
Val data length: 780
Test data length: 1170


## Model fine-tuning

In [27]:
set_seed(0)

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_enc.classes_)
)

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at yikuan8/Clinical-BigBird and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
print(torchinfo.summary(model))

Layer (type:depth-idx)                                            Param #
BigBirdForSequenceClassification                                  --
├─BigBirdModel: 1-1                                               --
│    └─BigBirdEmbeddings: 2-1                                     --
│    │    └─Embedding: 3-1                                        38,674,944
│    │    └─Embedding: 3-2                                        3,145,728
│    │    └─Embedding: 3-3                                        1,536
│    │    └─LayerNorm: 3-4                                        1,536
│    │    └─Dropout: 3-5                                          --
│    └─BigBirdEncoder: 2-2                                        --
│    │    └─ModuleList: 3-6                                       85,054,464
│    └─Linear: 2-3                                                590,592
│    └─Tanh: 2-4                                                  --
├─BigBirdClassificationHead: 1-2                                

In [30]:
torch.backends.cuda.matmul.allow_tf32 = True

training_args = TrainingArguments(
    tf32=True,
    dataloader_num_workers=4,
    output_dir=out_path,          # output directory
    disable_tqdm=False,
    num_train_epochs=epochs,
    per_device_train_batch_size=bs,  # batch size per device during training
    per_device_eval_batch_size=bs,   # batch size for evaluation
    learning_rate=lr,
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_safetensors=False,
    seed=0
)

In [31]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=llm_utils.compute_metrics_text_class
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [32]:
start_time = time.time()

trainer.train()

end_time = time.time()

/common/lopezgg/miniconda3/envs/nlp/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/common/lopezgg/miniconda3/envs/nlp/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.273300,0.277598,93.700000,87.800000,93.700000,90.700000
2,0.257700,0.278590,93.700000,87.800000,93.700000,90.700000
3,0.260100,0.242870,93.700000,87.800000,93.700000,90.700000
4,0.248400,0.256597,93.700000,87.800000,93.700000,90.700000
5,0.228400,0.267538,94.000000,94.300000,94.000000,91.300000
6,0.214500,0.282535,93.600000,91.200000,93.600000,91.600000
7,0.184500,0.299724,93.500000,89.500000,93.500000,90.800000
8,0.132200,0.347351,93.100000,90.400000,93.100000,91.300000
9,0.092900,0.390013,92.300000,90.000000,92.300000,91.000000
10,0.057300,0.403652,92.400000,90.100000,92.400000,91.100000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [33]:
print("Total training time:", str(timedelta(seconds=end_time - start_time)))

Total training time: 0:42:13.499508


In [34]:
arr_train_logs = copy.deepcopy(trainer.state.log_history)

In [35]:
train_stats = arr_train_logs.pop()

In [36]:
print("Training stats:")
print(train_stats)

Training stats:
{'train_runtime': 2530.9132, 'train_samples_per_second': 7.693, 'train_steps_per_second': 1.284, 'total_flos': 4.126477313654784e+16, 'train_loss': 0.19493397052471453, 'epoch': 10.0, 'step': 3250}


In [37]:
assert len(arr_train_logs) == epochs * 2

arr_print_logs = []
for i in range(0, len(arr_train_logs), 2):
    arr_print_logs.append({**arr_train_logs[i], **arr_train_logs[i+1]})

df_print_logs = pd.DataFrame(
    arr_print_logs,
    index=range(1, epochs+1)
)

In [38]:
df_print_logs

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second
1,0.2733,0.395976,0.000018,1.0,325,0.277598,93.7,87.8,93.7,90.7,24.1918,32.242,5.374
2,0.2577,0.234533,0.000016,2.0,650,0.278590,93.7,87.8,93.7,90.7,23.7910,32.786,5.464
3,0.2601,0.444847,0.000014,3.0,975,0.242870,93.7,87.8,93.7,90.7,24.2150,32.211,5.369
4,0.2484,0.271251,0.000012,4.0,1300,0.256597,93.7,87.8,93.7,90.7,24.1653,32.278,5.380
5,0.2284,1.107215,0.000010,5.0,1625,0.267538,94.0,94.3,94.0,91.3,24.4880,31.852,5.309
6,0.2145,0.239137,0.000008,6.0,1950,0.282535,93.6,91.2,93.6,91.6,24.3691,32.008,5.335
7,0.1845,32.403591,0.000006,7.0,2275,0.299724,93.5,89.5,93.5,90.8,23.8053,32.766,5.461
8,0.1322,0.078235,0.000004,8.0,2600,0.347351,93.1,90.4,93.1,91.3,23.8627,32.687,5.448
9,0.0929,0.906776,0.000002,9.0,2925,0.390013,92.3,90.0,92.3,91.0,23.9984,32.502,5.417
10,0.0573,0.650666,0.000000,10.0,3250,0.403652,92.4,90.1,92.4,91.1,23.8835,32.659,5.443


# Evaluation

## Validation

In [39]:
val_preds = trainer.predict(val_dataset)

/common/lopezgg/miniconda3/envs/nlp/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you c

In [40]:
print("Performance on val set:", llm_utils.compute_metrics_text_class(val_preds))

Performance on val set: {'accuracy': 93.6, 'precision': 91.2, 'recall': 93.6, 'f1': 91.6}


In [41]:
arr_val_label_preds = label_enc.inverse_transform(val_preds[0].argmax(axis=1))

In [42]:
accuracy_score(
    y_true=df_val[f'{tnm_label}_label'].values,
    y_pred=arr_val_label_preds
)

0.9358974358974359

In [43]:
utils.calculate_performance(
    arr_gs=df_val[f'{tnm_label}_label'].values,
    arr_preds=arr_val_label_preds,
    arr_labels=label_enc.classes_,
    col_label=f"{tnm_label}_label",
    df_data=df_val,
    df_train_data=df_train
)

,label,precision,recall,f1,n_train,n_val
0,M0,0.942783,0.991792,0.966667,1821,731
1,M1,0.454545,0.102041,0.166667,126,49


We save the model predictions (probability values):

In [44]:
with open(
    os.path.join(out_preds_path, f"{tnm_label}_label_{model_name.split('/')[-1]}_val_preds.pkl"),
    'wb'
) as file:
    pickle.dump(val_preds[0], file)

## Test

In [45]:
test_preds = trainer.predict(test_dataset)

/common/lopezgg/miniconda3/envs/nlp/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you c

In [46]:
print("Performance on test set:", llm_utils.compute_metrics_text_class(test_preds))

Performance on test set: {'accuracy': 92.9, 'precision': 89.5, 'recall': 92.9, 'f1': 90.8}


In [47]:
arr_test_label_preds = label_enc.inverse_transform(test_preds[0].argmax(axis=1))

In [48]:
accuracy_score(
    y_true=df_test[f'{tnm_label}_label'].values,
    y_pred=arr_test_label_preds
)

0.9290598290598291

In [49]:
utils.calculate_performance(
    arr_gs=df_test[f'{tnm_label}_label'].values,
    arr_preds=arr_test_label_preds,
    arr_labels=label_enc.classes_,
    col_label=f"{tnm_label}_label",
    df_data=df_test,
    df_train_data=df_train
)

,label,precision,recall,f1,n_train,n_val
0,M0,0.939289,0.988139,0.963095,1821,1096
1,M1,0.235294,0.054054,0.087912,126,74


We save the model predictions (probability values):

In [50]:
with open(
    os.path.join(out_preds_path, f"{tnm_label}_label_{model_name.split('/')[-1]}_test_preds.pkl"),
    'wb'
) as file:
    pickle.dump(test_preds[0], file)